Langchain Expression Language

- LCEL is that any two runnables can be "chained" together into sequences.

1. Sequential Chain
2. Parallel Chain
3. Router Chain
4. Chain Runnables
5. Custom Chain (Runnable Sequence)

In [1]:
from dotenv import load_dotenv

# LangSmith 환경변수 사용 (로컬에선 랭스미스말고 LangFuse 같은거 쓸 것)
load_dotenv('./../.env')

True

In [2]:
# Sequential LCEL Chain
from langchain_ollama import ChatOllama
from langchain_core.prompts import(
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)

# 랭체인 오라마 사용
base_url = "http://localhost:11434"
model = "llama3.2:1b"

llm = ChatOllama(base_url=base_url, model=model)
llm

ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')

In [3]:
# 기존의 프롬프트 템플릿 사용법
system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')
question = HumanMessagePromptTemplate.from_template('Tell me about the {topics} in {points} points')

messages = [system, question]
template = ChatPromptTemplate(messages)

question_template = template.invoke({'school':'primary', 'topics':'solar system','points':5})

response = llm.invoke(question_template)
print(response.content)


Here's what I have to say:

1. The sun is at the center of our solar system, making up most of its mass.
2. It has eight planets: Mercury, Mars, Venus, Earth, Neptune, Uranus, Saturn, and Jupiter.
3. Each planet has a unique shape due to its composition.
4. The four inner planets (Mercury to Neptune) are rocky worlds with different atmospheres.
5. Jupiter is the largest planet in our solar system, a gas giant with massive storms.


In [4]:
# Sequential LCEL 사용법
system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')
question = HumanMessagePromptTemplate.from_template('Tell me about the {topics} in {points} points')

messages = [system, question]
template = ChatPromptTemplate(messages)

# | 로 합친 것을 invoke
chain = template | llm

response = chain.invoke({'school':'primary', 'topics':'solar system','points':5})
print(response.content)

Here are five key points about the solar system:

1. Our solar system consists of eight planets (Mercury to Neptune) with five other smaller bodies: dwarf planet Pluto, asteroids, comets, and moons.

2. The four inner planets (Mercury to Mars) are rocky worlds, while the outer planets (Jupiter to Neptune) are gas giants with no solid surface.

3. Jupiter is the largest planet in our solar system, with a massive size and strong gravitational pull that holds the other planets in their orbits.

4. Venus is often called Earth's twin due to its similar size and mass, but it has a thick atmosphere that traps heat, making it extremely hot on the surface.

5. Mars is a rocky world with a thin atmosphere, and NASA's Curiosity rover recently discovered evidence of ancient rivers and lakes, suggesting that Mars may have been habitable in the past.


In [5]:
from langchain_core.output_parsers import StrOutputParser

chain = template | llm | StrOutputParser()
response = chain.invoke({'school':'primary','topics':'solar system','points':5})
print(response)             # content 사용을 하지 않음 (str만 남음)

Here's a brief overview of the solar system:

1. The sun is at the center, making up about 99% of the total mass.
2. There are eight planets: Mercury, Mars, Venus, Earth, Neptune, Uranus, Saturn, and Jupiter.
3. Each planet has its unique features, like Earth's atmosphere and Moon, which orbits it.
4. The closest planet to the sun is Mercury, while the farthest is Neptune.
5. Pluto was previously considered a planet but was reclassified as a dwarf planet in 2006 by the International Astronomical Union.


In [6]:
### Chaining Runnables (Chain Multiple Runnables)
# 위에서 받은 답변을 요약해달라고 하기

analysis_prompt = ChatPromptTemplate.from_template('''analyze the following text: {response}
                                                   You need tell me that how difficult it is to understand.
                                                   Answer in one sentence only.
                                                    ''')
fact_check_chain = analysis_prompt | llm | StrOutputParser()
output = fact_check_chain.invoke({'response':response})
print(output)

The text has some complex vocabulary and abstract concepts, such as "unique features", "atmosphere", and "dwarf planet", making it challenging for non-experts to grasp.


In [8]:
composed_chain = {"response":chain} | analysis_prompt | llm | StrOutputParser()

output = composed_chain.invoke({'school':'phd', 'topics':'solar system','points':5})
print(output)

The text is moderately clear, but may be challenging for non-expert readers with limited knowledge of astronomy or the solar system's classification.


In [27]:
### Parallel LCEL Chain
# run multiple runnables in parallel.
# return dict

system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')
question = HumanMessagePromptTemplate.from_template('Tell me about the {topics} in {points} points')
message = [system, question]
template = ChatPromptTemplate(message)

fact_chain = template | llm | StrOutputParser()
# output = fact_chain.invoke({'school':'primary','topics':'solar system','points':2})
# print(output)

The sun is at the center of our solar system.

There are eight planets, two dwarf planets, and five other smaller bodies orbiting around it: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.


In [28]:
question = HumanMessagePromptTemplate.from_template('Write a poem on {topics} in {sentences} lines')
message = [system, question]
template = ChatPromptTemplate(message)

poem_chain = template | llm | StrOutputParser()
# output = poem_chain.invoke({'school':'primary','topics':'solar system','sentences':2})
# print(output)

Here's a short poem:

The sun at center, planets sway,
Orbiting it, each one in its way.


In [29]:
question = HumanMessagePromptTemplate.from_template('Explain the {topics} in {sentences} lines in {language}')
message = [system, question]
template = ChatPromptTemplate(message)

third_chain = template | llm | StrOutputParser()

In [30]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(fact=fact_chain, poem=poem_chain, korean=third_chain)

output = chain.invoke({'school':'primary','topics':'solar system', 'points':2, 'sentences':2, 'language':'Korean'})

print(output['fact'])
print('\n\n')
print(output['poem'])
print('\n\n')
print(output['korean'])

Here's what you need to know about the solar system:

• The Sun is at the center of our solar system, which includes eight planets and many smaller objects like moons, asteroids, and comets.
• The four outer planets (Jupiter, Saturn, Uranus, and Neptune) are gas giants with no solid surface, while Mercury and Venus are rocky planets with solid surfaces.



The sun is at the center, 
Planets orbit around it with care.



우주의 반지름이 약 149.6만 km이며, 가장 큰 우주체는 주성 planets (mercury, mars, saturn, jupiter)이다. 우주의 중력의 강さ가 varies하며, farthest planet은 Neptune이고, farthest distance에서 관측할 수 있는 farthest planet은 Pluto이다.


In [36]:
### chain router
prompt = """
Given the user review below, classify it as either being about `Positive` or `Negative`.
Do not respond with more than one word.

Review: {review}
Classification:
"""

template = ChatPromptTemplate.from_template(prompt)

chain = template | llm | StrOutputParser()

# review = "Thank you so much for providing such a great platform for learning. I am really happy with the service."
# review = "I am not happy with the service. It is not good."
review = "보기에는 조촐해보였지만 성능은 확실합니다."

chain.invoke({'review':review})


'Positive'

In [39]:
positive_prompt = """
You are expert in writing reply for positive reviews.
You need to encourage the user to share their experience on social media.
Review: {review}
Answer:
"""

positive_template = ChatPromptTemplate.from_template(positive_prompt)
positive_chain = positive_template | llm | StrOutputParser()

In [40]:
negative_prompt = """
You are expert in writing reply for negative reviews.
You need first to apologize for the inconvenience caused to the user.
You need to encourage the user to share their concern on following Email:'uu@kk.com'.
Review: {review}
Answer:
"""

negative_template = ChatPromptTemplate.from_template(negative_prompt)
negative_chain = negative_template | llm | StrOutputParser()

In [41]:
def rout(info):
    if 'positive' in info['sentiment'].lower():
        return positive_chain
    else:
        return negative_chain

In [43]:
rout({'sentiment':'negative'})

ChatPromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, template="\nYou are expert in writing reply for negative reviews.\nYou need first to apologize for the inconvenience caused to the user.\nYou need to encourage the user to share their concern on following Email:'uu@kk.com'.\nReview: {review}\nAnswer:\n"), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')
| StrOutputParser()

In [44]:
from langchain_core.runnables import RunnableLambda

full_chain = {'sentiment':chain, 'review': lambda x: x['review']} | RunnableLambda(rout)

In [46]:
# review = "Thank you so much for providing such a great platform for learning. I am really happy with the service."
review = "I am not happy with the service. It is not good."

output = full_chain.invoke({'review':review})
print(output)

Here's a potential response:

"I'm so sorry to hear that your experience with our service didn't meet your expectations. We apologize for any inconvenience or frustration this has caused, and we're truly sorry that you didn't find our services satisfactory.

We take all complaints seriously and would like to make things right. Please know that we're actively working on improving our services, and we value your feedback as an important part of this process. If there's anything specific you'd like to share or if you have any further concerns, I'd be more than happy to listen and help in any way I can.

If you need assistance with a refund or compensation for your time, please don't hesitate to reach out to us at [your email address]. We're committed to providing the best possible service, and we appreciate your input in helping us improve. You can reply to this message directly from our website, and we'll get back to you promptly.

Thank you for sharing your feedback with us. We hope to 